<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=87d55c6c5fedba766d402f57a8bd739a8430d36b2be14f6d596417a3e4c433ac
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 11:11:43
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 66.19 K (0.47%)
Current PnL: -19.84 L (-13.22%)
CY Booked + Current PnL: -8.40 L (-5.6%)
-------------------
Total profit:  1.79 L
Total loss:  -21.63 L
-------------------
Total Booked + Current PnL: 18.58 L (15.09%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.07%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.35 L (61.62%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.48%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.29,119266.0,18762.0,10340.0,-1.97,18.67,8.67,28.96,79.0,1.81,0.84,58.98,MH,ATH,METALS
14,BLUESTARCO,2080.00,11.62,55.0,X-MC,3.16,192630.0,27960.0,15372.0,1.80,16.98,7.98,26.31,89.0,1.82,1.36,26.32,X40N,NTT,AC
84,VOLTAS,1530.00,2.00,63.0,X-MC,1.11,213885.0,22143.0,15614.0,3.18,11.55,7.30,19.69,99.0,1.42,1.52,19.13,XY25,NTT,AC
77,TTKPRESTIG,770.00,93.81,40.0,M-SC,0.99,83473.0,-17304.0,17396.0,0.02,-17.17,20.84,0.09,245.0,-0.99,0.59,9.67,OX40N,NTT,DURABLES
19,CIPLA,1662.74,-21.15,52.0,X-LC,3.64,210350.0,5850.0,17438.0,1.47,2.86,8.29,11.39,10.0,0.34,1.49,12.07,X40N,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.1,-39.52,49.0,H-SC,17.02,91790.0,-10710.0,51311.0,5.63,-10.45,55.90,39.61,114.0,-0.21,0.65,12.10,AR,ATH,MISC
0,5PAISA,593.0,123.56,56.0,H-SC,7.48,151632.0,-66864.0,95058.0,5.58,-30.60,62.69,12.90,153.0,-0.70,1.07,15.42,OX40N,NTT,FINANCE
84,VOLTAS,1530.0,2.00,63.0,X-MC,1.11,213885.0,22143.0,15614.0,3.18,11.55,7.30,19.69,99.0,1.42,1.52,19.13,XY25,NTT,AC
67,SONACOMS,1006.0,-37.75,59.0,M-SC,13.34,77455.0,-23704.0,98592.0,3.06,-23.43,127.29,74.03,202.0,-0.24,0.55,9.22,AR,BTT,AUTO
21,COFFEEDAY,80.0,-43.66,56.0,L-SC,29.30,84480.0,-29069.0,69122.0,3.04,-25.60,81.82,35.27,268.0,-0.42,0.60,106.77,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,31.71,62.0,M-LC,7.75,215108.0,10032.0,102542.0,-3.46,4.89,47.67,54.89,52.0,0.10,1.52,30.17,X5K,ATH,METALS
72,TATAELXSI,9161.00,-23.71,47.0,H-MC,6.64,103294.0,-28541.0,70767.0,-2.45,-21.65,68.51,32.03,98.0,-0.40,0.73,15.70,OX40N,NTT,IT
60,SAIL,228.00,45.95,53.0,M-MC,12.72,229939.0,4977.0,162682.0,-2.15,2.21,70.75,74.53,192.0,0.03,1.63,35.18,XY24,BTT,STEEL
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.29,119266.0,18762.0,10340.0,-1.97,18.67,8.67,28.96,79.0,1.81,0.84,58.98,MH,ATH,METALS
7,ATULAUTO,844.00,3672.97,52.0,M-SC,5.76,169371.0,-27761.0,117594.0,-1.51,-14.08,69.43,45.57,236.0,-0.24,1.20,20.08,XY24,NTT,AUTO


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,61.0,H-SC,13.53,130167.0,-12312.0,31839.0,1.07,-8.64,24.46,13.70,163.0,-0.39,0.92,53.63,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.81,40.0,M-SC,0.99,83473.0,-17304.0,17396.0,0.02,-17.17,20.84,0.09,245.0,-0.99,0.59,9.67,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,49.0,H-SC,1.93,221301.0,-48366.0,84692.0,0.22,-17.94,38.27,13.47,138.0,-0.57,1.57,12.53,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.71,47.0,H-MC,6.64,103294.0,-28541.0,70767.0,-2.45,-21.65,68.51,32.03,98.0,-0.40,0.73,15.70,OX40N,NTT,IT
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137425.0,-38478.0,80504.0,-0.80,-21.87,58.58,23.89,149.0,-0.48,0.97,18.68,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.95,53.0,M-MC,12.72,229939.0,4977.0,162682.0,-2.15,2.21,70.75,74.53,192.0,0.03,1.63,35.18,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,62.0,M-LC,7.75,215108.0,10032.0,102542.0,-3.46,4.89,47.67,54.89,52.0,0.10,1.52,30.17,X5K,ATH,METALS
86,WIPRO,420.00,-11.98,57.0,M-LC,5.69,154994.0,4049.0,105411.0,1.46,2.68,68.01,72.51,53.0,0.04,1.10,8.87,XR,NTT,IT
79,UNIONBANK,163.00,-13.18,56.0,M-LC,9.16,157895.0,17055.0,27442.0,0.75,12.11,17.38,31.59,66.0,0.62,1.12,41.81,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,68.0,H-MC,11.95,195172.0,15364.0,97040.0,0.88,8.54,49.72,62.52,88.0,0.16,1.38,45.54,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.29,119266.0,18762.0,10340.0,-1.97,18.67,8.67,28.96,79.0,1.81,0.84,58.98,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,68.0,H-MC,11.95,195172.0,15364.0,97040.0,0.88,8.54,49.72,62.52,88.0,0.16,1.38,45.54,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,62.0,M-LC,7.75,215108.0,10032.0,102542.0,-3.46,4.89,47.67,54.89,52.0,0.10,1.52,30.17,X5K,ATH,METALS
86,WIPRO,420.00,-11.98,57.0,M-LC,5.69,154994.0,4049.0,105411.0,1.46,2.68,68.01,72.51,53.0,0.04,1.10,8.87,XR,NTT,IT
37,INDIAMART,4810.62,-54.64,42.0,H-SC,8.73,126893.0,3557.0,128073.0,-0.75,2.88,100.93,106.72,119.0,0.03,0.90,26.94,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,DABUR,735.00,-12.66,34.0,X-MC,3.74,236438.0,-11630.0,120040.0,0.43,-4.69,50.77,43.70,102.0,-0.10,1.67,9.08,XY24,BTT,FMCG
5,ASIANPAINT,3465.66,-18.16,36.0,X-LC,5.55,206659.0,-45109.0,98328.0,0.51,-17.92,47.58,21.13,27.0,-0.46,1.46,11.16,X40,ATH,PAINTS
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53589.0,-11417.0,162343.0,2.81,-17.56,302.94,232.18,198.0,-0.07,0.38,2.82,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,37.0,X-MC,7.28,221700.0,-41665.0,155013.0,0.38,-15.82,69.92,43.04,84.0,-0.27,1.57,2.92,XY25,ATH,FMCG
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81949.0,-63211.0,139141.0,-0.46,-43.55,169.79,52.31,136.0,-0.45,0.58,9.69,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-5.33,42.0,X-MC,0.54,234702.0,-13029.0,88084.0,1.26,-5.26,37.53,30.30,92.0,-0.15,1.66,7.33,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,2.00,63.0,X-MC,1.11,213885.0,22143.0,15614.0,3.18,11.55,7.30,19.69,99.0,1.42,1.52,19.13,XY25,NTT,AC
73,TATAMOTORS,1065.00,-21.50,46.0,X-LC,1.61,248764.0,-27759.0,137840.0,0.62,-10.04,55.41,39.81,54.0,-0.20,1.76,19.22,XY24,NTT,AUTO
65,SIEMENS,4671.50,-1.76,47.0,H-LC,1.67,157735.0,-28360.0,75839.0,-0.75,-15.24,48.08,25.51,15.0,-0.37,1.12,16.24,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-18.72,50.0,X-MC,1.87,139830.0,3697.0,27435.0,0.37,2.72,19.62,22.86,91.0,0.13,0.99,15.67,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53589.0,-11417.0,162343.0,2.81,-17.56,302.94,232.18,198.0,-0.07,0.38,2.82,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,37.0,X-MC,7.28,221700.0,-41665.0,155013.0,0.38,-15.82,69.92,43.04,84.0,-0.27,1.57,2.92,XY25,ATH,FMCG
83,VBL,671.64,-20.34,43.0,X-LC,10.30,286909.0,-28933.0,141590.0,0.85,-9.16,49.35,35.67,5.0,-0.20,2.03,3.42,X40N,ATH,FMCG
42,ITC,452.00,-39.19,45.0,X-LC,2.41,196554.0,-3584.0,24471.0,0.51,-1.79,12.45,10.44,4.0,-0.15,1.39,3.67,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.27,40.0,X-MC,4.95,180744.0,-19762.0,116923.0,0.11,-9.86,64.69,48.45,90.0,-0.17,1.28,4.23,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53589.0,-11417.0,162343.0,2.81,-17.56,302.94,232.18,198.0,-0.07,0.38,2.82,XY24,NTT,MISC
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81949.0,-63211.0,139141.0,-0.46,-43.55,169.79,52.31,136.0,-0.45,0.58,9.69,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-35.01,43.0,X-SC,4.42,97234.0,-31436.0,76737.0,-0.07,-24.43,78.92,35.20,219.0,-0.41,0.69,11.60,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.72,51.0,X-SC,1.44,108223.0,6219.0,37705.0,0.89,6.10,34.84,43.06,122.0,0.16,0.77,24.26,X40N,ATH,MISC
33,HONAUT,58357.33,-22.51,48.0,X-SC,2.13,109200.0,-18738.0,65869.0,0.44,-14.65,60.32,36.84,143.0,-0.28,0.77,11.79,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,52.0,X-LC,12.15,285469.0,-60487.0,128718.0,-0.81,-17.48,45.09,19.73,1.0,-0.47,2.02,5.14,X40,ATH,IT
40,INFY,2275.00,-17.09,58.0,X-LC,7.78,322695.0,9807.0,161896.0,0.38,3.13,50.17,54.87,3.0,0.06,2.29,9.99,X40,BTT,IT
42,ITC,452.00,-39.19,45.0,X-LC,2.41,196554.0,-3584.0,24471.0,0.51,-1.79,12.45,10.44,4.0,-0.15,1.39,3.67,X40,NTT,FMCG
83,VBL,671.64,-20.34,43.0,X-LC,10.30,286909.0,-28933.0,141590.0,0.85,-9.16,49.35,35.67,5.0,-0.20,2.03,3.42,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,46.0,H-LC,7.50,248760.0,-12859.0,132067.0,0.70,-4.92,53.09,45.57,7.0,-0.10,1.76,5.85,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.30,84480.0,-29069.0,69122.0,3.04,-25.60,81.82,35.27,268.0,-0.42,0.60,106.77,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,42.0,H-SC,10.37,90705.0,-7275.0,28880.0,0.38,-7.42,31.84,22.05,152.0,-0.25,0.64,31.85,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.29,119266.0,18762.0,10340.0,-1.97,18.67,8.67,28.96,79.0,1.81,0.84,58.98,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,61.0,H-SC,13.53,130167.0,-12312.0,31839.0,1.07,-8.64,24.46,13.70,163.0,-0.39,0.92,53.63,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,66.0,M-SC,15.43,147177.0,-2043.0,146220.0,1.53,-1.37,99.35,96.62,208.0,-0.01,1.04,59.28,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,68.86,68.0,H-SC,9.12,155611.0,-23460.0,89196.0,2.66,-13.10,57.32,36.71,135.0,-0.26,1.10,54.40,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-185.23,66.0,M-SC,15.43,147177.0,-2043.0,146220.0,1.53,-1.37,99.35,96.62,208.0,-0.01,1.04,59.28,XY25,NTT,FINANCE
32,HINDZINC,730.22,31.71,62.0,M-LC,7.75,215108.0,10032.0,102542.0,-3.46,4.89,47.67,54.89,52.0,0.10,1.52,30.17,X5K,ATH,METALS
26,GREENPANEL,537.00,233.98,60.0,M-SC,4.68,149182.0,-33896.0,112871.0,0.21,-18.51,75.66,43.14,230.0,-0.30,1.06,40.33,XY24,NTT,MISC
50,NATIONALUM,244.55,-40.35,65.0,H-MC,14.29,119266.0,18762.0,10340.0,-1.97,18.67,8.67,28.96,79.0,1.81,0.84,58.98,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.15
1,25,43.39
2,50,75.18


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.99
LC    32.28
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.89
X40      14.62
X40N     11.25
XR       11.00
XY25     10.43
AR        8.16
OX40N     7.66
X200      1.74
MH        1.61
X5K       1.52
SR        1.10
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.46
X-LC    22.91
X-MC    17.18
M-SC    12.55
H-LC     4.62
H-MC     4.60
X-SC     4.52
M-LC     3.74
M-MC     1.63
L-SC     1.46
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.74,-5.71,40.56
IT,12.85,-15.91,78.68
FINANCE,10.54,-13.22,67.89
MISC,6.92,-17.79,83.06
BANKS,6.37,-10.80,69.08
PAINTS,5.47,-19.55,37.37
ELECTRICAL,5.31,-9.73,48.22
INSURANCE,3.78,-3.45,42.13
AC,3.57,3.79,12.31


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3212910.0,22
XR,1357549.0,14
AR,1178878.0,9
X40,775216.0,10
X40N,645964.0,9
OX40N,564997.0,10
XY25,465534.0,7
SR,265855.0,2
X5K,102542.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3404719.0,25
M-SC,1272945.0,15
X-MC,1107601.0,12
X-LC,1005513.0,13
X-SC,542557.0,6
H-MC,358525.0,4
H-LC,287673.0,3
L-SC,255830.0,3
M-LC,235395.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1200961.0      6
           AR         807532.0      5
           XR         796650.0      7
M-SC       XY24       684649.0      6
X-MC       XY24       589839.0      4
X-LC       X40        483639.0      5
X-SC       X40N       303477.0      4
H-SC       SR         265855.0      2
           OX40N      258658.0      4
X-MC       X40        214840.0      4
X-LC       X40N       210192.0      3
H-LC       AR         207906.0      2
X-LC       XY24       204616.0      2
H-MC       XY24       180378.0      1
X-MC       XY25       170627.0      2
L-SC       XR         167674.0      2
M-SC       AR         163440.0      2
M-MC       XY24       162682.0      1
X-SC       XY24       162343.0      1
M-SC       OX40N      147416.0      4
           XY25       146220.0      1
X-MC       X40N       132295.0      2
M-SC       XR         131220.0      2
X-LC       XY25       107066.0      3
M-LC       XR         105411.0      1
           X5K        102542.0      1
H-MC       XR          97040.0      1
L-SC       OX40N       88156.0      1
H-LC       X200        79767.0      1
X-SC       X40         76737.0      1
H-SC       MH          75063.0      1
H-MC       OX40N       70767.0      1
L-MC       XR          59554.0      1
L-LC       XY25        41621.0      1
M-LC       XY24        27442.0      1
H-MC       MH          10340.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
